In [ ]:
# REQUIRES INSTALLATION OF https://github.com/johli/seqprop

In [ ]:
import keras
from keras.models import Sequential, Model, load_model

from keras.layers import Dense, Dropout, Activation, Flatten, Input, Lambda, GlobalMaxPooling1D, concatenate
from keras.layers import Conv2D, MaxPooling2D, Conv1D, MaxPooling1D, LSTM, ConvLSTM2D, GRU, BatchNormalization, LocallyConnected2D, Permute
from keras.layers import Concatenate, Reshape, Softmax, Conv2DTranspose, Embedding, Multiply
from keras.callbacks import ModelCheckpoint, EarlyStopping, Callback
from keras import regularizers
from keras import backend as K
import keras.losses

import tensorflow as tf
from tensorflow.python.framework import ops

import isolearn.keras as iso

import numpy as np

import tensorflow as tf
import logging
logging.getLogger('tensorflow').setLevel(logging.ERROR)

import pandas as pd

import os
import pickle
import numpy as np

import scipy.sparse as sp
import scipy.io as spio

import matplotlib.pyplot as plt

import isolearn.keras as iso

from seqprop.visualization import *
from seqprop.generator import *
from seqprop.predictor import *
from seqprop.optimizer import *

import warnings
warnings.simplefilter("ignore")

class IdentityEncoder(iso.SequenceEncoder) :
    
    def __init__(self, seq_len, channel_map) :
        super(IdentityEncoder, self).__init__('identity', (seq_len, len(channel_map)))
        
        self.seq_len = seq_len
        self.n_channels = len(channel_map)
        self.encode_map = channel_map
        self.decode_map = {
            nt: ix for ix, nt in self.encode_map.items()
        }
    
    def encode(self, seq) :
        encoding = np.zeros((self.seq_len, self.n_channels))
        
        for i in range(len(seq)) :
            if seq[i] in self.encode_map :
                channel_ix = self.encode_map[seq[i]]
                encoding[i, channel_ix] = 1.

        return encoding
    
    def encode_inplace(self, seq, encoding) :
        for i in range(len(seq)) :
            if seq[i] in self.encode_map :
                channel_ix = self.encode_map[seq[i]]
                encoding[i, channel_ix] = 1.
    
    def encode_inplace_sparse(self, seq, encoding_mat, row_index) :
        raise NotImplementError()
    
    def decode(self, encoding) :
        seq = ''
    
        for pos in range(0, encoding.shape[0]) :
            argmax_nt = np.argmax(encoding[pos, :])
            max_nt = np.max(encoding[pos, :])
            seq += self.decode_map[argmax_nt]

        return seq
    
    def decode_sparse(self, encoding_mat, row_index) :
        raise NotImplementError()

In [2]:
def load_saved_predictor(model_path, library_context=None) :
    
    n_filters = 600
    filt_sizes = [25,11,7]
    n_dense = 64
    dropout_rate = 0.1
    
    sequence_input = Input(shape=(145, 4),name="pat_input")  
    convs = [None]*len(filt_sizes)
    
    for i in range(len(filt_sizes)):
        conv1           = Conv1D(n_filters, filt_sizes[i], padding='same', activation='linear', name = "pat_conv_" + str(i) + "_copy", trainable=False)(sequence_input)
        batchnorm1      = BatchNormalization(axis=-1,name = "pat_batchnorm_" + str(i) + "_copy", trainable=False)(conv1)
        relu1           = Activation('relu',name = "pat_relu_" + str(i) + "_copy")(batchnorm1)
        convs[i]        = Dropout(dropout_rate,name = "pat_dropout_" + str(i) + "_copy")(GlobalMaxPooling1D(name = "pat_pool_" + str(i) + "_copy")(relu1))
    
    concat1           = concatenate(convs,name="pat_concat_layer_copy")

    dense           = Dense(n_dense,activation='relu',name="pat_dense_copy", trainable=False)(concat1)
    output          = Dense(2,activation='linear',name="pat_output_copy", trainable=False)(dense)

    saved_model = Model(inputs=sequence_input,outputs=output)
    saved_model.compile(optimizer=keras.optimizers.Adam(lr=0.0002, beta_1=0.9, beta_2=0.999), loss="mse")
    

    saved_model.load_weights(model_path)

    def _initialize_predictor_weights(predictor_model, saved_model=saved_model) :
        
        #Load pre-trained model
        predictor_layers = ["pat_conv_0", "pat_conv_1", "pat_conv_2", "pat_batchnorm_0", 
                       "pat_batchnorm_1", "pat_batchnorm_2", "pat_dense", "pat_output"]
            
        for layer in predictor_layers:
            predictor_model.get_layer(layer).set_weights(saved_model.get_layer(layer + "_copy").get_weights())


    def _load_predictor_func(sequence_input) :
        #DragoNN parameters
        seq_length = 145
        n_filters = 600
        filt_sizes = [25,11,7]
        n_dense = 64
        dropout_rate = 0.1
        
        seq_input_shape = (seq_length, 4, 1)
        n_tasks = 1

        #Define model layers
        permute_input = Lambda(lambda x: x[..., -1])
        
        permuted_input = permute_input(sequence_input)
        for i in range(len(filt_sizes)):
            conv1           = Conv1D(n_filters, filt_sizes[i], padding='same', activation='linear',
                                     name = "pat_conv_" + str(i), trainable=False)(permuted_input)
            batchnorm1      = BatchNormalization(axis=-1,name = "pat_batchnorm_" + str(i), trainable=False)(conv1)
            relu1           = Activation('relu',name = "pat_relu_" + str(i))(batchnorm1)
            convs[i]        = Dropout(dropout_rate,name = "pat_dropout_" + str(i))(GlobalMaxPooling1D(name = "pat_pool_" + str(i))(relu1))
        
        concat1           = concatenate(convs,name="pat_concat_layer")
    
        dense           = Dense(n_dense,activation='relu',name="pat_dense", trainable=False)(concat1)
        output          = Dense(2,activation='linear',name="pat_output", trainable=False)(dense)

        #Execute functional model definition
        predictor_inputs = []
        predictor_outputs = [output]

        return predictor_inputs, predictor_outputs, _initialize_predictor_weights

    return _load_predictor_func

In [3]:
def load_predictor_model(model_path) :
    
    n_filters = 600
    filt_sizes = [25,11,7]
    n_dense = 64
    dropout_rate = 0.1
    
    sequence_input = Input(shape=(145, 4),name="pat_input")  
    convs = [None]*len(filt_sizes)
    
    for i in range(len(filt_sizes)):
        conv1           = Conv1D(n_filters, filt_sizes[i], padding='same', activation='linear', name = "pat_conv_" + str(i) + "_copy", trainable=False)(sequence_input)
        batchnorm1      = BatchNormalization(axis=-1,name = "pat_batchnorm_" + str(i) + "_copy", trainable=False)(conv1)
        relu1           = Activation('relu',name = "pat_relu_" + str(i) + "_copy")(batchnorm1)
        convs[i]        = Dropout(dropout_rate,name = "pat_dropout_" + str(i) + "_copy")(GlobalMaxPooling1D(name = "pat_pool_" + str(i) + "_copy")(relu1))
    
    concat1           = concatenate(convs,name="pat_concat_layer_copy")

    dense           = Dense(n_dense,activation='relu',name="pat_dense_copy", trainable=False)(concat1)
    output          = Dense(2,activation='linear',name="pat_output_copy", trainable=False)(dense)

    saved_model = Model(inputs=sequence_input,outputs=output)
    saved_model.compile(optimizer=keras.optimizers.Adam(lr=0.0002, beta_1=0.9, beta_2=0.999), loss="mse")
    

    saved_model.load_weights(model_path)
    return saved_model

In [4]:
acgt_encoder = IdentityEncoder(145, {'A':0, 'C':1, 'G':2, 'T':3})

In [11]:
def get_punish_margin_conv_activity(activity_margin=1.) :
    
    def _penalty(conv_out) :
        
        total_conv_out = K.abs(K.sum(conv_out, axis=-2))
        
        margin_cost = K.switch(total_conv_out < K.constant(activity_margin, shape=(1,)), K.zeros_like(total_conv_out), total_conv_out - K.constant(activity_margin, shape=(1,)))
        
        return K.mean(margin_cost, -1)
    
    return _penalty

#Define target isoform loss function
def get_earthmover_loss(target_output_ixs, pwm_start=0, pwm_end=70, pwm_target_bits=1.8, pwm_entropy_weight=0.0, conv_1_penalty=0.0, conv_1_margin=1.0, conv_2_penalty=0.0, conv_2_margin=1.0, conv_3_penalty=0.0, conv_3_margin=1.0) :
    
    punish_c = 0.0
    punish_g = 0.0
    
    entropy_mse = get_margin_entropy(pwm_start=pwm_start, pwm_end=pwm_end, min_bits=pwm_target_bits)
    
    punish_c_func = get_punish_c(pwm_start=pwm_start, pwm_end=pwm_end)
    punish_g_func = get_punish_g(pwm_start=pwm_start, pwm_end=pwm_end)
    

    fitness_target = 3.0 ### THIS CAN BE CHANGED, REALISTICALLY SHOULD HAVE BEEN AN ARGUMENT NOT HARD-CODED BUT I WAS A BABY GRAD STUDENT
    
    def loss_func(predictor_outputs) :
        pwm_logits, pwm, sampled_pwm, pred_score = predictor_outputs

        #Specify costs

        #### THIS SHOULD ALSO HAVE BEEN A PARAMETER BUT THE SUBTRACTION ORDER DETERMINES WHETHER H2K OR K2H IS OPTIMIZED AND IS HARD-CODED HERE
        fitness_loss = K.mean(K.maximum(-pred_score[..., 0] + pred_score[..., 1]+fitness_target, K.zeros_like(pred_score[..., 0])), axis=1)
        
        seq_loss = 0.0
        seq_loss += punish_c * K.mean(punish_c_func(sampled_pwm), axis=0)
        seq_loss += punish_g * K.mean(punish_g_func(sampled_pwm), axis=0)
        
        entropy_loss = pwm_entropy_weight * entropy_mse(pwm)
        
        #Compute total loss
        total_loss = fitness_loss + seq_loss + entropy_loss

        return K.reshape(K.sum(total_loss, axis=0), (1,))
    
    def val_loss_func(predictor_outputs) :
        pwm_logits, pwm, sampled_pwm, pred_score = predictor_outputs

        #Specify costs
        fitness_loss = K.mean(K.maximum(-pred_score[..., 1] + pred_score[..., 0]+fitness_target, K.zeros_like(pred_score[..., 0])), axis=1)
    
        seq_loss = 0.0
        seq_loss += punish_c * K.mean(punish_c_func(sampled_pwm), axis=0)
        seq_loss += punish_g * K.mean(punish_g_func(sampled_pwm), axis=0)
        
        entropy_loss = pwm_entropy_weight * entropy_mse(pwm)
        
        #Compute total loss
        total_loss = fitness_loss + seq_loss + entropy_loss

        return K.reshape(K.mean(total_loss, axis=0), (1,))
    
    return loss_func, val_loss_func


def get_nop_transform() :
    
    def _transform_func(pwm) :
        
        return pwm
    
    return _transform_func

class ValidationCallback(Callback):
    def __init__(self, val_name, val_loss_model, val_steps) :
        self.val_name = val_name
        self.val_loss_model = val_loss_model
        self.val_steps = val_steps
        
        self.val_loss_history = []
        
        #Track val loss
        self.val_loss_history.append(self.val_loss_model.predict(x=None, steps=self.val_steps)[0])
    
    def on_batch_end(self, batch, logs={}) :
        #Track val loss
        val_loss_value = self.val_loss_model.predict(x=None, steps=self.val_steps)[0]
        self.val_loss_history.append(val_loss_value)

#Function for running SeqProp on a set of objectives to optimize
def run_seqprop(target_output_ixs_list, sequence_templates, loss_funcs, val_loss_funcs, transform_funcs,model_path, n_sequences=1, n_samples=1, n_valid_samples=1, eval_mode='sample', normalize_logits=False, n_epochs=10, steps_per_epoch=100) :
    
    n_objectives = len(sequence_templates)
    
    seqprop_predictors = []
    valid_monitors = []
    train_histories = []
    valid_histories = []
    
    for obj_ix in range(n_objectives) :
        print("Optimizing objective " + str(obj_ix) + '...')
        
        sequence_template = sequence_templates[obj_ix]
        loss_func = loss_funcs[obj_ix]
        val_loss_func = val_loss_funcs[obj_ix]
        transform_func = transform_funcs[obj_ix]
        target_output_ixs = target_output_ixs_list[obj_ix]
        
        #Build Generator Network
        _, seqprop_generator = build_generator(seq_length=len(sequence_template), n_sequences=n_sequences, n_samples=n_samples, sequence_templates=[sequence_template * n_sequences], batch_normalize_pwm=normalize_logits, pwm_transform_func=transform_func, validation_sample_mode='sample')
        _, valid_generator = build_generator(seq_length=len(sequence_template), n_sequences=n_sequences, n_samples=n_valid_samples, sequence_templates=[sequence_template * n_sequences], batch_normalize_pwm=normalize_logits, pwm_transform_func=None, validation_sample_mode='sample', master_generator=seqprop_generator)
        for layer in valid_generator.layers :
            #if 'policy' not in layer.name :
            layer.name += "_valversion"
        
        #Build Predictor Network and hook it on the generator PWM output tensor
        _, seqprop_predictor = build_predictor(seqprop_generator, load_saved_predictor(model_path, library_context=None), n_sequences=n_sequences, n_samples=n_samples, eval_mode=eval_mode)
        _, valid_predictor = build_predictor(valid_generator, load_saved_predictor(model_path, library_context=None), n_sequences=n_sequences, n_samples=n_valid_samples, eval_mode='sample')
        for layer in valid_predictor.layers :
            if '_valversion' not in layer.name :# and 'policy' not in layer.name :
                layer.name += "_valversion"
        
        #Build Loss Model (In: Generator seed, Out: Loss function)
        _, loss_model = build_loss_model(seqprop_predictor, loss_func)
        _, valid_loss_model = build_loss_model(valid_predictor, val_loss_func)
        
        #Specify Optimizer to use
        opt = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999)

        #Compile Loss Model (Minimize self)
        loss_model.compile(loss=lambda true, pred: pred, optimizer=opt)

        def get_logit(p) :
            return np.log(p / (1. - p))
        
#         #Specify callback entities
#         #measure_func = lambda pred_outs: np.mean(get_logit(np.expand_dims(pred_outs[0], axis=0) if len(pred_outs[0].shape) <= 2 else pred_outs[0]), axis=0)
#         measure_func = lambda pred_outs: np.expand_dims(np.mean(np.expand_dims(pred_outs[0][..., target_output_ixs], axis=0) if len(pred_outs[0].shape) <= 2 else pred_outs[0][..., target_output_ixs], axis=(0, -1)), axis=-1)
        
#         #train_monitor = FlexibleSeqPropMonitor(predictor=seqprop_predictor, plot_on_train_end=False, plot_every_epoch=False, track_every_step=True, measure_func=measure_func, measure_name='Activity', plot_pwm_start=500, plot_pwm_end=700, sequence_template=sequence_template, plot_pwm_indices=np.arange(n_sequences).tolist(), figsize=(12, 1.0))
#         valid_monitor = FlexibleSeqPropMonitor(predictor=valid_predictor, plot_on_train_end=True, plot_every_epoch=False, track_every_step=True, measure_func=measure_func, measure_name='Activity', plot_pwm_start=0, plot_pwm_end=145, sequence_template=sequence_template, plot_pwm_indices=np.arange(n_sequences).tolist(), figsize=(12, 1.0))
        
#         train_history = ValidationCallback('loss', loss_model, 1)
#         valid_history = ValidationCallback('val_loss', valid_loss_model, 1)
        
#         callbacks =[
#             #EarlyStopping(monitor='loss', min_delta=0.001, patience=5, verbose=0, mode='auto'),
#             valid_monitor,
#             train_history,
#             valid_history
#         ]
        
        #Fit Loss Model
        _ = loss_model.fit(
            [], np.ones((1, 1)), #Dummy training example
            epochs=n_epochs,
            steps_per_epoch=steps_per_epoch,
#             callbacks=callbacks
        )
        
#         valid_monitor.predictor = None
#         train_history.val_loss_model = None
#         valid_history.val_loss_model = None
        
        seqprop_predictors.append(seqprop_predictor)
#         valid_monitors.append(valid_monitor)
#         train_histories.append(train_history)
#         valid_histories.append(valid_history)

    return seqprop_predictors#, valid_monitors, train_histories, valid_histories


In [20]:
import random

def set_seed(seed_value) :
    # 1. Set the `PYTHONHASHSEED` environment variable at a fixed value
    os.environ['PYTHONHASHSEED']=str(seed_value)

    # 2. Set the `python` built-in pseudo-random generator at a fixed value
    random.seed(seed_value)

    # 3. Set the `numpy` pseudo-random generator at a fixed value
    np.random.seed(seed_value)

    # 4. Set the `tensorflow` pseudo-random generator at a fixed value
    tf.set_random_seed(seed_value)

    # 5. Configure a new global `tensorflow` session
    session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
    sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
    K.set_session(sess)


In [ ]:


#Run SeqProp Optimization

#Specify file path to pre-trained predictor network

save_dir = os.path.join(os.getcwd(), '')
model_dir = f"{os.getcwd()}\\predictor_models\\single_predictors"
seq_file_prefix = "h2k_seqprop_single"
model_basename = 'wide'

seq_template = 'N' * 145

# rand_seed = 14755

#Number of PWMs to generate per objective
n_sequences = 10
#Number of One-hot sequences to sample from the PWM at each grad step
n_samples = 1
#Number of epochs per objective to optimize
n_epochs = 1
#Number of steps (grad updates) per epoch
steps_per_epoch = 200
#Number of One-hot validation sequences to sample from the PWM
n_valid_samples = 10

experiment_name_list = ['Sampled-IN']
eval_mode_list = ['sample']
normalize_logits_list = [True]

n_models = 10
for model_ix in range(n_models):
    model_name = f"{model_basename}_{model_ix}.h5"
    model_path = os.path.join(model_dir, model_name)

    result_dict = {}

    for experiment_name, eval_mode, normalize_logits in zip(experiment_name_list, eval_mode_list, normalize_logits_list) :

        print("Experiment name = " + str(experiment_name))
        print("Eval mode = " + str(eval_mode))
        print("Normalize logits = " + str(normalize_logits))

        K.clear_session()

    #     set_seed(rand_seed)

        target_output_ixs = [
            [0,1]
        ]

        sequence_templates = [
            seq_template
        ]

        losses, val_losses = zip(*[
            get_earthmover_loss(
                target_output_ixs[0],
                pwm_start=0,
                pwm_end=145,
                pwm_target_bits=1.8,
                pwm_entropy_weight=0.0
            )
        ])

        transforms = [
            None
        ]

        seqprop_predictors = run_seqprop(target_output_ixs, sequence_templates, losses, val_losses,
                                         transforms, model_path, n_sequences, n_samples, n_valid_samples,
                                         eval_mode, normalize_logits, n_epochs, steps_per_epoch)

        seqprop_predictor = seqprop_predictors[0]#, valid_monitors[0], train_histories[0], valid_histories[0]

        #Retrieve optimized PWMs and predicted cleavage distributionns
        _, optimized_pwm, _, _ = seqprop_predictor.predict(x=None, steps=1)

        consensus_seqs = []
        for i in range(optimized_pwm.shape[0]) :
            consensus_seq = ''
            for j in range(optimized_pwm.shape[1]) :
                max_nt_ix = np.argmax(optimized_pwm[i, j, :, 0])
                if max_nt_ix == 0 :
                    consensus_seq += 'A'
                elif max_nt_ix == 1 :
                    consensus_seq += 'C'
                elif max_nt_ix == 2 :
                    consensus_seq += 'G'
                elif max_nt_ix == 3 :
                    consensus_seq += 'T'

            consensus_seqs.append(consensus_seq)

        result_dict[experiment_name] = {
            'consensus_seqs' : consensus_seqs
        }
        
        
        with open(seq_file_prefix + "_sequences.csv", "at") as f:
            seqs = np.array(result_dict['Sampled-IN']['consensus_seqs'])
            onehots = np.array([acgt_encoder.encode(seq) for seq in seqs])
            predictor = load_predictor_model(model_path)
            score_preds = predictor.predict(x=onehots)
            h2k_score = score_preds[:,0] - score_preds[:,1]
            sort_index = np.argsort(h2k_score)[::-1]
            seqs = seqs[sort_index]
            h2k_score = h2k_score[sort_index]

            f.write(f"{seqs[0]}, {h2k_score[0]}, seqprop, {model_basename}_{model_ix}\n")
